In [2]:
pip install python-crfsuite

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
pip install numpy


[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: sklearn is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install sklearn-crfsuite

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install python-crfsuite openpyxl nltk


     ------------------------------------- 250.0/250.0 kB 40.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import openpyxl
import nltk

def extract_features(tokens):
    features = []
    for i in range(len(tokens)):
        word = tokens[i]
        features.append({
            'word': word,
         #   'pos': nltk.pos_tag([word])[0][1],
        })
    return features

def extract_labels(data):
    return [label for _, label in data]


In [14]:
def read_data_from_excel(file_path):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    data = []
    for row in sheet.iter_rows(values_only=True):
        token = row[0]
        label = row[1]
        data.append((token, label))
    return data


In [15]:
import pycrfsuite

def train_crf_model(train_data, feature_extractor, label_extractor, model_path):
    trainer = pycrfsuite.Trainer(verbose=False)
    for tokens, labels in train_data:
        features = feature_extractor(tokens)
        trainer.append(features, labels)

    trainer.set_params({
        'c1': 1.0,   # coefficient for L1 penalty
        'c2': 1e-3,  # coefficient for L2 penalty
        'max_iterations': 100,  # maximum number of iterations
        'feature.possible_transitions': True  # include transitions that are possible, but not observed
    })
    
    trainer.train(model_path)


In [16]:
def test_crf_model(test_data, feature_extractor, label_extractor, model_path):
    tagger = pycrfsuite.Tagger()
    tagger.open(model_path)

    predictions = []
    for tokens, _ in test_data:
        features = feature_extractor(tokens)
        pred_labels = tagger.tag(features)
        predictions.append(pred_labels)

    true_labels = [label_extractor(data) for data in test_data]

    return true_labels, predictions


In [23]:
train_file = 'Nepali_Romanized_Sentiment.xlsx'
model_path = 'sentiment_model.crfsuite'

# Load and preprocess the training data
train_data = read_data_from_excel(train_file)
print(len(train_data))
# Split the data into tokens and labels
train_tokens, train_labels = zip(*train_data)
print(len(train_labels))

# Perform any necessary preprocessing on the tokens and labels

# Train the model
train_crf_model(zip(train_tokens, train_labels), extract_features, extract_labels, model_path)


3596
3596


TypeError: object of type 'NoneType' has no len()